In [1]:
#!pip install -U sparkmagic
!pip install -qU langchain-google-community\[gmail\] langchain-aws langchain langchain-community

In [2]:
pip install -qU  google-api-python-client google-auth-oauthlib google-auth-httplib2 

Note: you may need to restart the kernel to use updated packages.


In [ ]:

import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_PROJECT"] = "pr-drab-pay-91"



In [58]:
from langchain_google_community.gmail.utils import (
    build_resource_service,
    get_google_credentials,
    build_gmail_service
)
from langchain_google_community import GmailToolkit

In [59]:
import webbrowser

webbrowser.register('chrome', None, webbrowser.GenericBrowser('/usr/bin/google-chrome'))

In [9]:
from google_auth_oauthlib.flow import Flow
from google.oauth2.credentials import Credentials
import os

# Define your scopes and client secrets file
scopes = ['https://mail.google.com/']
client_secrets_file = 'client_secret_704484678251-3ln7sr0ul540e3b6k11gj69lsqjf1i39.apps.googleusercontent.com.json'
token_file = 'token.json'

# Create the OAuth flow manually
flow = Flow.from_client_secrets_file(
    client_secrets_file=client_secrets_file,
    scopes=scopes,
    redirect_uri='urn:ietf:wg:oauth:2.0:oob'  # This is the out-of-band redirect URI
)

# Generate authorization URL
auth_url, _ = flow.authorization_url(prompt='consent')

print("Please go to this URL and authorize the application:")
print(auth_url)

# Ask user to paste the authorization code
code = input("Enter the authorization code here: ")
flow.fetch_token(code=code)

# Get credentials
creds = flow.credentials

# Save credentials for future use
with open(token_file, 'w') as token:
    token.write(creds.to_json())



Please go to this URL and authorize the application:
https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=704484678251-3ln7sr0ul540e3b6k11gj69lsqjf1i39.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fmail.google.com%2F&state=QYH7YaEpCuAKkqOQD5gZ5AQzmSc1fQ&prompt=consent&access_type=offline


Enter the authorization code here:  4/1Ab32j91Bz67PVYEyssdyMewcJ0VfhmHGXpFnFz6S9HwKsTv91ctyGJgYCYI


In [60]:
from langchain_aws.chat_models.bedrock import ChatBedrock

api_resource = build_gmail_service(credentials=creds)
toolkit = GmailToolkit(api_resource=api_resource)

In [61]:
tools = toolkit.get_tools()
tools

[GmailCreateDraft(api_resource=<googleapiclient.discovery.Resource object at 0x7f379377eec0>),
 GmailSendMessage(api_resource=<googleapiclient.discovery.Resource object at 0x7f379377eec0>),
 GmailSearch(api_resource=<googleapiclient.discovery.Resource object at 0x7f379377eec0>),
 GmailGetMessage(api_resource=<googleapiclient.discovery.Resource object at 0x7f379377eec0>),
 GmailGetThread(api_resource=<googleapiclient.discovery.Resource object at 0x7f379377eec0>)]

In [62]:
llm = ChatBedrock(
    model_id="openai.gpt-oss-120b-1:0",
    model_kwargs={"temperature": 0.001},
    verbose=True
)

In [63]:
SYSTEM_PROMPT = """
You are an intelligent email assistant designed to help users manage their email communications. 
You can create drafts, send emails, and delete emails using the provided tools. 
Agent should not stop without completing the operation
Follow these rules:

1. Drafting Emails:
   - When the user provides email content (recipients, subject, body),
     use the draft creation tool and create a draft email in gmail box.

2. Sending Emails:
   - You can send emails directly or from saved drafts.
   - Notify the user once the email is successfully sent.
   - don't stop without sending email.
   - email should be send from logged in mail box

3. Search emails:
   - You can search email from my inbox and send me the details.

General Behavior:
- Be polite, clear, and concise.
- Log actions where appropriate for traceability.
- Don't stop until the operation specified is completed.
- create the email in the mail box
- Search email and tool calling should happen. 
- Do't add your pretrained details 
"""



In [64]:
from langchain.agents import create_agent


agent_executor = create_agent(llm, tools, system_prompt=SYSTEM_PROMPT)

In [71]:
example_query = "Create a draft email to nitya.cse.hit@gmail.com thanking them for coffee."

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values"
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Create a draft email to nitya.cse.hit@gmail.com thanking them for coffee.
================================== Ai Message ==================================

<reasoning>We need to create a draft email. Use tool? There's a draft creation tool. We need to call it. The user wants a draft email to nitya.cse.hit@gmail.com thanking them for coffee. We need subject and body. Probably subject: Thank you for the coffee. Body: Dear Nitya, Thank you for the coffee... etc.

We must call the tool. The tool name? Not given explicitly but likely something like create_draft. We need to infer from system? Not provided. Usually in these tasks, there are tool definitions. But none shown. Might be hidden. We need to attempt to call a tool. The instruction: "When the user provides email content (recipients, subject, body), use the draft creation tool and create a draft email in gmail box."

Thus we need to call a tool. The tool

In [69]:

from langsmith import Client

client = Client()
projects = client.list_projects()
print([p.name for p in projects])


[]


In [70]:

project = client.create_project(
    project_name="pr-new-collard-32",
    description="Tracing project for email agent",
    upsert=True  # Ensures it updates if the project already exists
)

print("Project created:", project)


Project created: id=UUID('492244d4-3b55-4c3b-bd70-2c579b28bd18') start_time=datetime.datetime(2025, 11, 2, 9, 7, 43, 172327, tzinfo=datetime.timezone.utc) end_time=None description='Tracing project for email agent' name='pr-new-collard-32' extra=None tenant_id=UUID('8f6416a3-a71b-4ae0-99bc-6d90f31886b9') reference_dataset_id=None
